In [1]:
GOOGLE_CLOUD_PROJECT="o-epm-gcp-by-meetup1-ml-t1iylu"
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/oleg/Projects/EPAM/creds.json
%cd /Users/oleg/Projects/EPAM/taxi/ml-specialization-demo1/tfx-pipeline

from pipeline import configs, components
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tfx
context = InteractiveContext(
    pipeline_root=configs.PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(
              '/Users/oleg/Projects/EPAM/taxi/metadata1.db'
    )
)

env: GOOGLE_CLOUD_PROJECT=o-epm-gcp-by-meetup1-ml-t1iylu
env: GOOGLE_APPLICATION_CREDENTIALS=/Users/oleg/Projects/EPAM/creds.json
/Users/oleg/Projects/EPAM/taxi/ml-specialization-demo1/tfx-pipeline


In [3]:
example_gen=components.example_gen()
context.run(example_gen)

statistics_gen=components.statistics_gen(example_gen=example_gen)
context.run(statistics_gen)

schema_gen = components.schema_gen(statistics_gen=statistics_gen)
context.run(schema_gen)

transform = components.transform(example_gen=example_gen,schema_gen=schema_gen)
context.run(transform)

trainer = components.trainer_vertex(
    example_gen=example_gen, schema_gen=schema_gen, transform=transform)
context.run(trainer)

model_resolver = components.model_resolver()
context.run(model_resolver)

evaluator = components.evaluator(example_gen=example_gen, trainer=trainer, model_resolver=model_resolver)
context.run(evaluator)

pusher = components.pusher_vertex(trainer=trainer, evaluator=evaluator)
context.run(pusher)
pusher.outputs




ValueError: push_destination is required unless a custom_executor_spec is supplied that does not require it.

In [5]:
from tfx.dsl.components.common.importer import Importer
model = Importer(
    source_uri='gs://chicago-taxi-ml-demo-1/tfx_pipeline_output/tfx-pipeline-ml/Trainer/model/1020',
    artifact_type=tfx.types.standard_artifacts.Model,
    reimport=True)

context.run(model)

model=model.outputs['result']

In [6]:
from tfx.extensions.google_cloud_ai_platform.pusher.executor import ENABLE_VERTEX_KEY, VERTEX_REGION_KEY, VERTEX_CONTAINER_IMAGE_URI_KEY
from tfx.extensions.google_cloud_ai_platform.pusher import executor as ai_platform_pusher_executor
from tfx.dsl.components.base import executor_spec
from tfx.proto import pusher_pb2

pusher = tfx.components.Pusher(
        model=model.outputs['result'],
        custom_executor_spec=executor_spec.ExecutorClassSpec(ai_platform_pusher_executor.Executor),
        custom_config={
            ENABLE_VERTEX_KEY: True,
            VERTEX_REGION_KEY: 'us-central1',
            VERTEX_CONTAINER_IMAGE_URI_KEY: 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-5:latest',
            ai_platform_pusher_executor.SERVING_ARGS_KEY: {
                'model_name': 'try_pusher',  # '-' is not allowed.
                'project_id': GOOGLE_CLOUD_PROJECT,
                'regions': ['us-central1'],
                'endpoint_name': 'my_model_endpoint',
                'min_replica_count': 1,
                'max_replica_count': 2,
                'machine_type': 'n1-standard-2'
            },
        },
        # push_destination=pusher_pb2.PushDestination(
        #     filesystem=pusher_pb2.PushDestination.Filesystem(
        #         base_directory="gs://chicago-taxi-ml-demo-1/tfx_pipeline_output/tfx-pipeline-ml/Pusher/model/1020"
        #     )
        # )
).with_id('Pusher')
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 97
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 112
        type_id: 14
        uri: "gs://chicago-taxi-ml-demo-1/tfx_pipeline_output/tfx-pipeline-ml/Pusher/pushed_model/97"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "projects/1012336990068/locations/us-central1/models/3651636445518495744"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
from pipeline.components import example_gen, tuner


_example_gen = example_gen()
context.run(_example_gen)
exampes = _example_gen.outputs['examples']

_tuner = tuner(examples=exampes, base_model=model)
context.run(_tuner)
_tuner.outputs


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying model.py -> build/lib
installing to /var/folders/2f/96xj0q5139vghz17xfkfghpw0000gn/T/tmp96dpjfb8
running install
running install_lib
copying build/lib/model.py -> /var/folders/2f/96xj0q5139vghz17xfkfghpw0000gn/T/tmp96dpjfb8
running install_egg_info
running egg_info
creating tfx_user_code_HyperparametersTuner.egg-info
writing tfx_user_code_HyperparametersTuner.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_HyperparametersTuner.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_HyperparametersTuner.egg-info/top_level.txt
writing manifest file 'tfx_user_code_HyperparametersTuner.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_HyperparametersTuner.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_HyperparametersTuner.egg-info/SOURCES.txt'
Copying tfx_user_code_HyperparametersTuner.egg-info to /var/folders/2f/96xj0q5139vghz17xfkfghpw0000gn/T/tmp96

Traceback (most recent call last):
  File "/Users/oleg/Projects/EPAM/taxi/ml-specialization-demo1/venv/lib/python3.8/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/oleg/Projects/EPAM/taxi/ml-specialization-demo1/venv/lib/python3.8/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/oleg/Projects/EPAM/taxi/ml-specialization-demo1/venv/lib/python3.8/site-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . i

RuntimeError: Job 'projects/o-epm-gcp-by-meetup1-ml-t1iylu/jobs/tfx_tuner_20210906213647' did not succeed.  Detailed response {'jobId': 'tfx_tuner_20210906213647', 'trainingInput': {'scaleTier': 'CUSTOM', 'masterType': 'standard', 'workerType': 'standard', 'workerCount': '2', 'region': 'us-central1', 'masterConfig': {'imageUri': 'gcr.io/o-epm-gcp-by-meetup1-ml-t1iylu/tfx-pipeline-ml', 'containerCommand': ['python', '-m', 'tfx.scripts.run_executor', '--executor_class_path', 'tfx.extensions.google_cloud_ai_platform.tuner.executor._WorkerExecutor', '--inputs', '{"examples": [{"artifact": {"id": "116", "type_id": "19", "uri": "gs://chicago-taxi-ml-demo-1/tfx_pipeline_output/tfx-pipeline-ml/CsvExampleGen/examples/102", "properties": {"split_names": {"string_value": "[\\"train\\", \\"eval\\"]"}}, "custom_properties": {"payload_format": {"string_value": "FORMAT_TF_EXAMPLE"}, "state": {"string_value": "published"}, "span": {"int_value": "0"}, "file_format": {"string_value": "tfrecords_gzip"}, 